In [1]:
import spacy

from spacy.tokens import Token
from spacy.symbols import NOUN, PROPN, PRON, ADJ, ADV, ADP, VERB

nlp = spacy.load("en_core_web_md")
nlp.add_pipe("merge_entities")

def get_left_edge(word):

    # print(word.left_edge, ':', word, '=>', word.doc[word.left_edge.i : word.i + 1])
    nouns = [NOUN, PROPN, PRON]
    if word.pos not in nouns:
        return word.i
    if word.left_edge.pos == PRON:
        return word.i
    return word.left_edge.i

def get_right_edge(word):
    
    nouns = [NOUN, PROPN, PRON]
    if word.pos not in nouns:
        return word.i

    next_token = next(word.rights, False)
    
    if next_token and next_token.pos in (ADJ, ADV):
        return next_token.i
    
    if next_token and next_token.pos == ADP:        
        for right_token in next_token.rights:
            if right_token.dep_ == "pobj" and right_token.head.i == next_token.i:
                return right_token.i
            
    return word.i

Token.set_extension("get_left_edge", method=get_left_edge, force=True)
Token.set_extension("get_right_edge", method=get_right_edge, force=True)
print(Token.has_extension("get_left_edge"))
print(Token.has_extension("get_right_edge"))

True
True


In [2]:
from LoadText import load_text

bush_filename = "20010920-Address to Joint Session of Congress Following 911 Attacks.txt"
laden_filename = "19960823-Declaration of Jihad Against the Americans Occupying the Land of the Two Holiest Sites.txt"

bush = load_text(bush_filename)
binLaden = load_text(laden_filename)
print(f'Bush: {len(bush)} words')
print(f'bin Laden: {len(binLaden)} words')

Bush: 17321 words
bin Laden: 48741 words


In [3]:
%load_ext autoreload
%autoreload 2

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


from dependency_patterns import _object, _subject, _subject_passive, _adjective
from dependency_patterns import coordination_deps, preposition_deps
from spacy.matcher import DependencyMatcher
from DependencyMatchers import CustomDependencyMatcher
import pandas as pd
from spacy import displacy

# dependency_matcher = DependencyMatcher(nlp.vocab)

travel_verbs = {"LEMMA": {"IN": ["traffic", "smuggle", "bring", "travel", "take"]}}
live_verbs = {"LEMMA": {"IN": ["traffic", "bring", "travel", "take"]}}
mode_lemma = {"LEMMA": {"IN": ["on"]}}
to_lemma = {"LEMMA": "to"}
from_lemma = {"LEMMA": "from"}
by_lemma = {"LEMMA": "by"}
VERB = {"POS": {"IN": ["VERB"]}}
AUX = {"POS": {"IN": ["AUX"]}}
GPE = {"ENT_TYPE": "GPE"}
agent_dep = {"DEP": {"IN": ["agent"]}}

patterns = [
    {
        "pattern_name": "SingleVerbClause",
        "pattern":
            [
                {"RIGHT_ID": "PREDICATE", "RIGHT_ATTRS": VERB},
                {"LEFT_ID": "PREDICATE", "REL_OP": ">", "RIGHT_ID": "SUBJECT", "RIGHT_ATTRS": _subject},
                {"LEFT_ID": "PREDICATE", "REL_OP": ">", "RIGHT_ID": "OBJECT", "RIGHT_ATTRS": _object},
            ],
        "category": "hasLinkTo", "inverse": "isLinkedTo"
    },
    {
        "pattern_name": "hasAttribute",
        "pattern":
            [
                {"RIGHT_ID": "PREDICATE", "RIGHT_ATTRS": AUX},
                {"LEFT_ID": "PREDICATE", "REL_OP": ">", "RIGHT_ID": "SUBJECT", "RIGHT_ATTRS": _subject},
                {"LEFT_ID": "PREDICATE", "REL_OP": ">", "RIGHT_ID": "MODIFIER", "RIGHT_ATTRS": _adjective},
            ],
        "category": "hasAttribute", "inverse": "isLinkedTo"
    },
    {
        "pattern_name": "SinglePassivePrepositionalPhrase",
        "pattern":
            [
                {"RIGHT_ID": "PREDICATE", "RIGHT_ATTRS": VERB},
                {"LEFT_ID": "PREDICATE", "REL_OP": ">", "RIGHT_ID": "SUBJECT", "RIGHT_ATTRS": _subject_passive},
                {"LEFT_ID": "PREDICATE", "REL_OP": ".", "RIGHT_ID": "PREPOSITION", "RIGHT_ATTRS": coordination_deps},
                {"LEFT_ID": "PREPOSITION", "REL_OP": ">", "RIGHT_ID": "OBJECT", "RIGHT_ATTRS": preposition_deps},
            ],
        "category": "hasLinkTo", "inverse": "isLinkedTo"
    },
    {
        "pattern_name": "DoublePrepositionalPhrase",
        "pattern":
            [
                {"RIGHT_ID": "PREDICATE", "RIGHT_ATTRS": VERB},
                {"LEFT_ID": "PREDICATE", "REL_OP": ">", "RIGHT_ID": "SUBJECT", "RIGHT_ATTRS": _subject},
                {"LEFT_ID": "PREDICATE", "REL_OP": ".", "RIGHT_ID": "PREPOSITION", "RIGHT_ATTRS": coordination_deps},
                {"LEFT_ID": "PREPOSITION", "REL_OP": ">", "RIGHT_ID": "OBJECT", "RIGHT_ATTRS": preposition_deps},
                {"LEFT_ID": "PREPOSITION", "REL_OP": ">", "RIGHT_ID": "PREPOSITION2", "RIGHT_ATTRS": coordination_deps},
                {"LEFT_ID": "PREPOSITION2", "REL_OP": ">", "RIGHT_ID": "OBJECT2", "RIGHT_ATTRS": preposition_deps}
            ],
        "category": "hasLinkTo", "inverse": "isLinkedTo"
    },
]

extract_clauses = CustomDependencyMatcher(nlp, patterns)
org_doc = nlp(bush)

matches = []

for sent in org_doc.sents:

    doc = nlp(sent.text)

    clauses = extract_clauses(doc)
    if clauses:
        displacy.render(doc, style='dep')
        display(pd.DataFrame(clauses))
        print('-----')

Added:  SingleVerbClause
Added:  hasAttribute
Added:  SinglePassivePrepositionalPhrase
Added:  DoublePrepositionalPhrase


,RULE,CATEGORY,PREDICATE,SUBJECT,OBJECT
0,SingleVerbClause,hasLinkTo,seen,We,it
1,SingleVerbClause,hasLinkTo,rushed,who,terrorists


-----


,RULE,CATEGORY,PREDICATE,SUBJECT,OBJECT
0,SingleVerbClause,hasLinkTo,help,you,me


-----


...

-----


,RULE,CATEGORY,PREDICATE,SUBJECT,OBJECT
0,SingleVerbClause,hasLinkTo,meet,we,violence with patient justice


-----


,RULE,CATEGORY,PREDICATE,SUBJECT,OBJECT
0,SingleVerbClause,hasLinkTo,grant,God,us
1,SingleVerbClause,hasLinkTo,grant,God,wisdom


-----
